In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import mysql.connector
import networkx as nx
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon,MultiPolygon,LineString,MultiLineString,LinearRing,Point
from shapely import symmetric_difference
from shapely.ops import nearest_points
from shapely.wkt import loads
import scipy
import time
import numpy as np
import random
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt
import osmnx as ox
import glob
from itertools import combinations,product
from copy import deepcopy
import os
from lib_contour_lines import *
import mysql.connector


In [2]:

cnx = mysql.connector.connect(user='spirz', 
                              password='this_is_my_PASSWORD_m8',
                              host='localhost',
                              database='dem_from_contours_lines',
                              autocommit=True)
cursor = cnx.cursor(buffered=True,dictionary=True)
contours_lines_table_name='contours_lines'
tree_edges_table_name="tree_edges"
osm_nodes_table_name="osm_nodes"
osm_intersections_table_name="osm_intersections"
2

2

In [3]:
t1=time.time()
G=build_graph(cursor)
t2=time.time()
print(t2-t1)

0.6687564849853516


In [4]:
t1=time.time()
cmd="SELECT osm_id,contour_line_id,ST_asText(geometry) AS geometry FROM %s WHERE contour_line_id IS NOT NULL"%osm_nodes_table_name
cursor.execute(cmd)
nodes_data=pd.DataFrame(cursor.fetchall())
nodes_data['geometry']=nodes_data['geometry'].apply(lambda pt:loads(pt))
nodes_data=nodes_data.set_index('osm_id',drop=False)
nodes_data=gpd.GeoDataFrame(nodes_data,geometry='geometry',crs='epsg:4326')
nodes_data['old_geometry']=nodes_data['geometry'].copy()
crs=nodes_data.estimate_utm_crs()
nodes_data=nodes_data.to_crs(crs)
nodes_data['depth']=nodes_data['contour_line_id'].apply(lambda contour_line_id:G.nodes()[contour_line_id]['depth'])
t2=time.time()
print(t2-t1)

1.5330610275268555


In [27]:
max_nb_neighbors=float('inf')
t1=time.time()
nodes_data['elevation']=None
contour_line_ids=nodes_data['contour_line_id'].unique()
contour_lines_ids=sorted(contour_line_ids,key=lambda contour_line_id:len(list(G.neighbors(contour_line_id))))
contour_lines_ids=contour_lines_ids[::-1]
print('sorted')
for k,root_contour_line_id in enumerate(contour_lines_ids):
    if (k+1)%10==0:
        print('%i/%i'%(k+1,len(contour_line_ids)))
    neighbor_contour_line_ids=list(G.neighbors(root_contour_line_id))
    print('%i neighbors'%len(neighbor_contour_line_ids))
    if 0<len(neighbor_contour_line_ids)<max_nb_neighbors:
        t3=time.time()
        local_contours_line_ids=[root_contour_line_id]+neighbor_contour_line_ids
        local_contour_data=get_nodes_data(cursor,local_contours_line_ids)
        local_contour_data=local_contour_data.to_crs(crs)
        local_contour_data=local_contour_data.set_index('id')
        t4=time.time()
        print('setting up df took %f'%(t4-t3))
        local_nodes_data=nodes_data[nodes_data.contour_line_id==root_contour_line_id].copy()
        local_nodes_data['elevation']=local_nodes_data['geometry'].apply(lambda osm_pt:get_osm_node_elevation(osm_pt,local_contour_data))
        nodes_data.loc[nodes_data.contour_line_id==root_contour_line_id,'elevation']=local_nodes_data.loc[:,'elevation']
    
        t5=time.time()
        print('finding affine basis took %f'%(t5-t4))
        print('intermediate time %f'%(t5-t1))
t2=time.time()
print(t2-t1)

sorted
607 neighbors
setting up df took 0.680909


/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


finding affine basis took 245.713747
intermediate time 246.401835
228 neighbors
setting up df took 0.890879


/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


finding affine basis took 138.127610
intermediate time 385.420396
205 neighbors
setting up df took 0.861248


...

setting up df took 0.080683
finding affine basis took 0.012678
intermediate time 2188.586254
1 neighbors
setting up df took 0.075946
finding affine basis took 0.009959
intermediate time 2188.672197
3540/4337
1 neighbors
setting up df took 0.012632
finding affine basis took 0.003058
intermediate time 2188.687935
1 neighbors
setting up df took 0.004794
finding affine basis took 0.003003
intermediate time 2188.695760
1 neighbors
setting up df took 0.010048
finding affine basis took 0.004141
intermediate time 2188.709975
1 neighbors
setting up df took 0.005808
finding affine basis took 0.003415
intermediate time 2188.719240
1 neighbors
setting up df took 0.005377
finding affine basis took 0.003364
intermediate time 2188.728009
1 neighbors
setting up df took 0.005918
finding affine basis took 0.004780
intermediate time 2188.738733
1 neighbors
setting up df took 0.005280
finding affine basis took 0.010991
intermediate time 2188.755036
1 neighbors
setting up df took 0.004982
finding affine ba

/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


setting up df took 0.006549
finding affine basis took 0.005217
intermediate time 2188.784852
3550/4337
1 neighbors
setting up df took 0.005835
finding affine basis took 0.004206
intermediate time 2188.794935
1 neighbors
setting up df took 0.008905
finding affine basis took 0.003791
intermediate time 2188.807672
1 neighbors
setting up df took 0.007017
finding affine basis took 0.005783
intermediate time 2188.820507
1 neighbors
setting up df took 0.005997
finding affine basis took 0.004077
intermediate time 2188.830618
1 neighbors
setting up df took 0.014687
finding affine basis took 0.017554
intermediate time 2188.862892
1 neighbors
setting up df took 0.005811
finding affine basis took 0.003673
intermediate time 2188.872411
1 neighbors
setting up df took 0.005514
finding affine basis took 0.005685
intermediate time 2188.883640
1 neighbors
setting up df took 0.004843
finding affine basis took 0.004360
intermediate time 2188.892872
1 neighbors
setting up df took 0.005553
finding affine ba

/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
/home/pierre/anaconda3/conda/envs/pygmt/lib/python3.11/site-packages/shapely/linear.py:90: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)


finding affine basis took 0.004163
intermediate time 2189.412920
1 neighbors
setting up df took 0.006773
finding affine basis took 0.004057
intermediate time 2189.423796
1 neighbors
setting up df took 0.008972
finding affine basis took 0.005230
intermediate time 2189.438036
1 neighbors
setting up df took 0.007138
finding affine basis took 0.003970
intermediate time 2189.449182
1 neighbors
setting up df took 0.008143
finding affine basis took 0.004329
intermediate time 2189.461686
0 neighbors
0 neighbors
0 neighbors
0 neighbors
3590/4337
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
3600/4337
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
3610/4337
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
3620/4337
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neighbors
0 neigh

In [29]:
nodes_data=nodes_data[nodes_data.elevation==nodes_data.elevation]
diffs=nodes_data['elevation']-nodes_data['contour_line_id'].apply(lambda node:G.nodes()[node]['elevation'])
np.min(diffs),np.mean(diffs),np.median(diffs),np.max(diffs)
                                                                  

(-3.804320174625772, 2.5403660817182185, 2.565415188861948, 4.999979206454384)

In [39]:
UPDATE_ELEVATION=True
if UPDATE_ELEVATION:
    t1=time.time()
    cmd="""CREATE TEMPORARY TABLE temp (
    `osm_id` BIGINT NOT NULL,
    `elevation` FLOAT NOT NULL

    );"""
    cursor.execute(cmd)

    cmd="INSERT INTO temp(`osm_id`,`elevation`) VALUES %s"%',\n'.join(['(%i,%f)'%(osm_id,row['elevation']) for osm_id,row in nodes_data.iterrows()])
    cursor.execute(cmd)

    cmd=""" UPDATE %s JOIN temp
    ON %s.osm_id=temp.osm_id
    SET %s.elevation=temp.elevation

    ;"""%(osm_nodes_table_name,osm_nodes_table_name,osm_nodes_table_name)
    cursor.execute(cmd)
    
    cmd="DROP TABLE temp"
    cursor.execute(cmd)
    t2=time.time()
    print(t2-t1)
2

5.063480854034424


2